In [21]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
from sklearn.linear_model import LinearRegression
plt.style.use("ggplot")
import catboost as cb 
import pandas as pd
import numpy as np

# import lightgbm as lgb 
# model = ltb.LGBMRegressor()

In [22]:
sample = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/AI Hackflow/SampleSubmission.csv') 
test = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/AI Hackflow/Test.csv')
train = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/AI Hackflow/Train.csv')

### Formule

In [37]:
test['energy_unit'] = round((test['EUI'] * test['Total Floors Area'] ) / test['Number of Floors'])

In [38]:
sample = pd.read_csv('C:/Users/tun20/Desktop/Hackathons/AI Hackflow/submission1 (2).csv') 
sample['prediction_label']  = test['energy_unit']
sample.to_csv('laziz_raby.csv', index=False)

### Pre-processing

In [23]:

def prepross(data):
    wwr = []
    
    for x in data['WWR']: 
        test_map = x.replace('(' , '').replace(')' , '')
        tupe = tuple(map(float, test_map.split(',')))
        wwr.append(mean(tupe))
    data[['Cooling', 'Lights']] = data['random load mesures'].str.extract(r'Cooling\': \'(?P<Cooling>[0-9\.]+):C\', \'Lights\': \'(?P<Lights>[0-9\.]+):C')
    data['Cooling']=data['Cooling'].astype(float)
    data['Lights']=data['Lights'].astype(float)
    data.drop(columns=['random load mesures','building','File'], inplace= True)
    data['WWR'] = wwr
    return data 
    
def feature_engineer(data): 
    data['energy_unit'] = (data['EUI'] * data['Total Floors Area'] ) / data['Number of Floors']
    data['energy_unit'] = [np.round(num, 9) for num in data['energy_unit']]
    data = data.drop(['EUI', 'Total Floors Area','Number of Floors'], axis = 1 )
    
    return data

In [24]:

train = prepross(train)
train = feature_engineer(train)
test = prepross(test)
test = feature_engineer(test)

In [25]:
train.columns

Index(['Cooling Setpoint', 'EUI', 'Cooling COP', 'Operating Hours', 'WWR',
       'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt',
       'Infiltration', 'Ground Floor Rt', 'Number of Floors', 'Occupancy',
       'Light Heat Gain', 'Windows Rt', 'Height', 'Heating COP',
       'Heating Setpoint', 'Town', 'Wall Rt', 'Start Time', 'windows g-value',
       'Roof Rt', 'Boiler Efficiency', 'Internal Mass', 'Permeability',
       'Total Floors Area', 'Operational Energy', 'Cooling', 'Lights',
       'energy_unit'],
      dtype='object')

In [26]:
train.drop(columns={'Cooling COP', 'Operating Hours', 'WWR',
       'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt',
       'Infiltration', 'Ground Floor Rt', 'Occupancy', 'Light Heat Gain',
       'Windows Rt', 'Height', 'Heating COP', 'Heating Setpoint', 'Town',
       'Wall Rt', 'Start Time', 'windows g-value', 'Roof Rt',
       'Boiler Efficiency', 'Internal Mass', 'Permeability',
       'Cooling', 'Lights'}, inplace= True)

In [27]:
test.drop(columns={'Cooling COP', 'Operating Hours', 'WWR',
       'Equipment Heat Gain', 'Internal Wall Rt', 'Internal Floor Rt',
       'Infiltration', 'Ground Floor Rt', 'Occupancy', 'Light Heat Gain',
       'Windows Rt', 'Height', 'Heating COP', 'Heating Setpoint', 'Town',
       'Wall Rt', 'Start Time', 'windows g-value', 'Roof Rt',
       'Boiler Efficiency', 'Internal Mass', 'Permeability',
       'Cooling', 'Lights'}, inplace= True)

### XGBoost

In [15]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train.drop('Operational Energy', axis=1), train['Operational Energy'], test_size=0.2, random_state=42)

In [17]:
model = xgb.XGBRegressor(objective='reg:squarederror',
                          n_estimators=1000,
                          learning_rate=0.05,
                          max_depth=6,
                          min_child_weight=1,
                          gamma=0,
                          subsample=0.8,
                          colsample_bytree=0.8,
                          nthread=4,
                          seed=42)

In [18]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=4, nthread=4, num_parallel_tree=1, predictor='auto',
             random_state=42, reg_alpha=0, ...)

In [19]:
y_pred = model.predict(X_test)

In [21]:
from sklearn.metrics import mean_squared_error as MSE
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, y_pred))

print("RMSE : % f" %(rmse))

RMSE :  264.254641


In [22]:
params = {
    'max_depth': range(3,10,2),
    'min_child_weight': range(1,6,2)
}
gsearch = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', n_jobs=4, iid=False, cv=5)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_, gsearch.best_score_)

c:\Users\tun20\.conda\envs\dlia\lib\site-packages\sklearn\model_selection\_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'max_depth': 5, 'min_child_weight': 3} -75532.87083694126


In [25]:
y_pred = gsearch.predict(X_test)

In [26]:
from sklearn.metrics import mean_squared_error as MSE
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, y_pred))

print("RMSE : % f" %(rmse))

RMSE :  241.115719


### Lasso

In [28]:
from sklearn.linear_model import Lasso

In [29]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train.drop('Operational Energy', axis=1), train['Operational Energy'], test_size=0.1, random_state=42)

In [30]:
lasso_reg = Lasso()

In [31]:
lasso_reg.fit(X_train,y_train)

Lasso()

In [32]:
y_pred =lasso_reg.predict(X_test)

In [33]:
from sklearn.metrics import mean_squared_error as MSE
 
# RMSE Computation
rmse = np.sqrt(MSE(y_test, y_pred))

print("RMSE : % f" %(rmse))

RMSE :  0.032864


In [12]:
submission = lasso_reg.predict()

TypeError: predict() missing 1 required positional argument: 'X'

### Catboost on the full dataset

In [28]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train.drop('Operational Energy', axis=1), train['Operational Energy'], test_size=0.2, random_state=42)

In [29]:
# Define the CatBoost model
model = cb.CatBoostRegressor(iterations=1000, 
                              learning_rate=0.05, 
                              depth=6, 
                              eval_metric='RMSE',
                              random_seed=42)

In [30]:
# Fit the CatBoost model
model.fit(X_train, y_train, verbose=False)

In [31]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [32]:
from sklearn.metrics import mean_squared_error as MSE

# RMSE Computation
rmse = np.sqrt(MSE(y_test, y_pred))

print("RMSE : % f" %(rmse))

RMSE :  1154.611214


### regression en personne

In [7]:
# x = train['Operational Energy']
# v =  (train['EUI'] * train['Total Floors Area'] ) / train['Number of Floors']
# rounded_vec = [np.round(num, 9) for num in v]


In [8]:
# falsy = {}
# for i in range(len(x)): 
#     if x[i] != rounded_vec[i] : 
#         falsy[f'index_{i}'] = [ x[i], rounded_vec[i]]
    
# len(falsy)

In [5]:

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as MSE
X = np.array(train['energy_unit'])
X = X.reshape(-1,1)
y = np.array(train['Operational Energy'])
lasso_reg = Lasso()

lasso_reg.fit(X,y)

y_pred_lass =lasso_reg.predict(np.array(X).reshape(-1,1))



  
# RMSE Computation
rmse = np.sqrt(MSE(y, y_pred_lass))

print("RMSE : % f" %(rmse))

RMSE :  0.000015


In [15]:
# X = np.array(train['energy_unit','Internal Maass'])
X = np.array(train['energy_unit'])
X = X.reshape(-1,1)
y = np.array(train['Operational Energy'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# model = cb.CatBoostRegressor(loss_function='RMSE')
reg = LinearRegression(fit_intercept=True ).fit(X, y)


In [16]:
(X - y ).sum() 

MemoryError: Unable to allocate 22.2 GiB for an array with shape (54600, 54600) and data type float64

In [17]:
from sklearn.metrics import mean_squared_error as MSE
pred = reg.predict(np.array(X_test).reshape(-1,1))
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))



ValueError: Found input variables with inconsistent numbers of samples: [10920, 283920]

In [207]:
result = lasso_reg.predict(np.array(test['energy_unit']).reshape(-1,1)) 


In [163]:
result/10000000000


array([ 73062.46341943, 106643.52589948,  88718.96574199, ...,
       228667.95442258,  60061.91806241, 191278.64959934])

In [208]:

sample_clean = pd.read_csv('/kaggle/input/submission-1-2/submission1 (2).csv')
sample_clean['prediction_label']  = result
sample_clean.to_csv('lasso_regression_version_2_rounded_decimals.csv', index=False)

In [ ]:
# X = train.drop(['building','File', 'Operational Energy', 'random load mesures'], axis = 1)
# y = train['Operational Energy']


model = cb.CatBoostRegressor(loss_function='RMSE' , depth=10, iterations= 1000, learning_rate = 0.1)
model.fit(X_train, y_train)
# optimized_cat = grid.best_estimator_

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
pred = model.predict(X_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))


In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(gird.best_score_)

In [ ]:
test[['Cooling', 'Lights']] = test['random load mesures'].str.extract(r'Cooling\': \'(?P<Cooling>[0-9\.]+):C\', \'Lights\': \'(?P<Lights>[0-9\.]+):C')
wwr = []
from statistics import mean
for x in test['WWR']:
    test_map = x.replace('(' , '').replace(')' , '')
    tupe = tuple(map(float, test_map.split(',')))
    wwr.append(mean(tupe)) 
test['WWR'] = wwr
test['Cooling'] = test['Cooling'].astype('float32')
test['Lights'] = test['Lights'].astype('float32')


In [ ]:
# test['Cooling'] = test['Cooling'].astype('float32')
# test['Lights'] = test['Lights'].astype('float32')
# test =  test.drop([ 'building','File' , 'random load mesures'], axis =1 )
result = model.predict(test)

In [ ]:
result.shape

In [ ]:
test_again = pd.read_csv("/kaggle/input/deepfdata/Test.csv")
# test_again['ID'] = test_again.apply(lambda x: f"Building_{x['building'].split('')[1]}_Town{x['Town']}", axis=1)
test_again.head() 
# test_again['ID'] = test_again.apply(lambda x : f"Building_{x['building'].split('')[1]}_Town{x['Town']}", axis = 1 )

In [ ]:
test_again = pd.read_csv("/kaggle/input/deepfdata/Test.csv")
test_again.drop(columns={'File', 'Cooling Setpoint', 'EUI', 'Cooling COP',
       'Operating Hours', 'WWR', 'Equipment Heat Gain', 'Internal Wall Rt',
       'Internal Floor Rt', 'Infiltration', 'Ground Floor Rt',
       'Number of Floors', 'Occupancy', 'Light Heat Gain', 'Windows Rt',
       'Height', 'Heating COP', 'Heating Setpoint', 'Town', 'Wall Rt',
       'Start Time', 'windows g-value', 'Roof Rt', 'Boiler Efficiency',
       'Internal Mass', 'Permeability', 'Total Floors Area',
       'random load mesures'}, inplace= True)

In [ ]:
test_again['Operational Energy'] = result

In [ ]:
test_again.to_csv('submission_1_xgb_simple.csv', index = False )

In [ ]:
test_again.rename(columns = {'building':'submission_id'}, inplace = True)

In [ ]:
test_again.to_csv('submission_1_xgb_simple.csv', index = False )

In [ ]:
sample_clean = pd.read_csv('/kaggle/input/submission-1-2/submission1 (2).csv')

In [ ]:
sample_clean['prediction_label']  = result

In [ ]:
sample_clean.to_csv('catboost_800mse_iterations.csv', index=False)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train) 
lgb_eval = lgb.Dataset(X_test, y_test, reference = lgb_train)

In [ ]:
params = {
    'task' : 'train',
    'boosting' : 'gbdt', 
    'objective' : 'regression', 
    'learning_rate' : 0.05 , 
    'metric' : {'l2','l1'}, 
    'verbose' : -1 
}

model = lgb.train(params, 
                 train_set = lgb_train,
                 valid_sets = lgb_eval , 
                 early_stopping_rounds = 40 )


In [ ]:
y_pred = model.predict(X_test) 
mse = MSE(y_test, y_pred) 
rmse = mse**(0.5)
print("RMSE", rmse)